# Assignment 1: Data Collection and Prompt Engineering (10%)

This assignment requires students to create a data set for training and evaluation of an SUTD chatbot for prospective students. The data set should contain documents about SUTD and question-answer pairs suitable for model training and evaluation.

In addition to the data set, students should build a first prototype using only prompt engineering and foundation models available via APIs.

Objectives:
- Collect and curate documents related to SUTD (programs, admissions, campus, scholarships, student life, FAQs, policies).
- Create a high-quality Q&A dataset suitable for training and evaluation.
- Build a prompt-engineered chatbot prototype using a foundation model API with [Amazon Bedrock — What is Bedrock?](https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-bedrock.html).
- Evaluate prototype responses against the curated Q&A dataset.

Deliverables:
- Data artifact: documents (raw + cleaned), Q&A pairs (JSONL), metadata (sources, timestamps, licenses).
- Notebook: end-to-end workflow (collection → cleaning → Q&A generation → prototype → evaluation).
- Short report: methodology, data sources, prompt design, evaluation summary.

Grading (10%):
- Data quality and coverage (3%)
- Q&A diversity, clarity, and correctness (3%)
- Prototype design and prompt engineering (2%)
- Evaluation thoroughness and analysis (2%)


# Setup & Environment

This notebook uses Python. If you use external APIs (e.g., AWS Bedrock), store credentials in environment variables and load them securely (do not hardcode keys).

Recommended packages:
- requests, beautifulsoup4, pandas, numpy, tqdm
- scikit-learn (optional for baseline retrieval)
- openai OR anthropic OR boto3 (choose one API path)
- rouge-score OR evaluate (optional, for metrics)

Tip: Use a virtual environment and a `.env` file or system keychain.

Example environment variables:
- OPENAI_API_KEY
- ANTHROPIC_API_KEY
- AWS_REGION + AWS credentials for Bedrock


# Data Collection

Collect documents about SUTD from official, public sources (admissions pages, program descriptions, campus life, FAQ, scholarship info, policies). Respect robots.txt and terms of use; avoid overloading servers, and cache downloads locally.

Suggested steps:
1. Identify seed URLs and a scope definition (which pages to include).
2. Fetch pages politely (rate limiting), parse text, and store raw HTML + cleaned text.
3. Track metadata: URL, title, section, timestamp, retrieval status, license.
4. Normalize and deduplicate content; segment long pages into sections.

Artifacts:
- data/raw/*.html
- data/processed/*.md or *.txt
- data/metadata.csv


**1. Fetching raw HTML from SUTD FAQ website**

To ground the chatbot in official university information, I collected content from the SUTD Undergraduate Admissions FAQ website.
The FAQ content spans 9 paginated pages, each containing multiple question-and-answer entries.
All source URLs were stored in data/seed_urls.txt



The pipeline is as follows:

seed_urls.txt --> fetch_html.py --> raw HTML files + metadata.csv

**HTML to readable text**

In [12]:
from bs4 import BeautifulSoup
import os

PROCESSED_DIR = "data/processed"
os.makedirs(PROCESSED_DIR, exist_ok=True)

In [18]:
from bs4 import BeautifulSoup
from pathlib import Path
import re

RAW_DIR = Path("data/raw")
OUT_DIR = Path("data/processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

SEPARATOR = "--------------"

def clean(s: str) -> str:
    s = re.sub(r"\s+\n", "\n", s)
    s = re.sub(r"\n{3,}", "\n\n", s)
    return s.strip()

def extract_faq_from_html(html: str):
    soup = BeautifulSoup(html, "html.parser")

    accordion = soup.select_one("section#accordion")
    if not accordion:
        return []

    qa_pairs = []

    # Each FAQ item contains an h6 (question) and a div.richText (answer)
    # We'll pair them by walking each question to its nearest following richText.
    for h6 in accordion.select("h6"):
        q = h6.get_text(" ", strip=True)
        q = clean(q)
        if not q:
            continue

        body = h6.find_parent()
        # search forward in the DOM for the first answer block
        ans_div = h6.find_next("div", class_="richText")
        if not ans_div:
            continue

        # Keep paragraphs + links as text
        a = ans_div.get_text("\n", strip=True)
        a = clean(a)

        # Guard against accidentally capturing empty/irrelevant blocks
        if a:
            qa_pairs.append((q, a))

    return qa_pairs

def write_qa_txt(qa_pairs, out_path: Path):
    parts = []
    for q, a in qa_pairs:
        parts.append(SEPARATOR)
        parts.append(q)
        parts.append("")        # empty line
        parts.append(a)
    out_text = "\n".join(parts).rstrip()  # keep trailing separator optional
    out_path.write_text(out_text, encoding="utf-8")

# Run over all downloaded HTML pages
for html_path in sorted(RAW_DIR.glob("*.html")):
    html = html_path.read_text(encoding="utf-8", errors="ignore")
    qa = extract_faq_from_html(html)

    out_path = OUT_DIR / f"{html_path.stem}_faq.txt"
    write_qa_txt(qa, out_path)

    print(f"{html_path.name} → {out_path.name} | extracted {len(qa)} Q&A")

admissions_undergraduate_faq_paged_1.html → admissions_undergraduate_faq_paged_1_faq.txt | extracted 10 Q&A
admissions_undergraduate_faq_paged_2.html → admissions_undergraduate_faq_paged_2_faq.txt | extracted 10 Q&A
admissions_undergraduate_faq_paged_3.html → admissions_undergraduate_faq_paged_3_faq.txt | extracted 10 Q&A
admissions_undergraduate_faq_paged_4.html → admissions_undergraduate_faq_paged_4_faq.txt | extracted 10 Q&A
admissions_undergraduate_faq_paged_5.html → admissions_undergraduate_faq_paged_5_faq.txt | extracted 10 Q&A
admissions_undergraduate_faq_paged_6.html → admissions_undergraduate_faq_paged_6_faq.txt | extracted 10 Q&A
admissions_undergraduate_faq_paged_7.html → admissions_undergraduate_faq_paged_7_faq.txt | extracted 10 Q&A
admissions_undergraduate_faq_paged_8.html → admissions_undergraduate_faq_paged_8_faq.txt | extracted 10 Q&A
admissions_undergraduate_faq_paged_9.html → admissions_undergraduate_faq_paged_9_faq.txt | extracted 3 Q&A


In [19]:
# Combine all 9 files into 1
from pathlib import Path

PROCESSED_DIR = Path("data/processed")
combined_path = PROCESSED_DIR / "sutd_undergrad_faq_all.txt"

faq_files = sorted(PROCESSED_DIR.glob("*_faq.txt"))

combined_text = "\n\n".join(f.read_text(encoding="utf-8") for f in faq_files)
combined_path.write_text(combined_text, encoding="utf-8")

print(f"Combined {len(faq_files)} files → {combined_path}")

Combined 9 files → data/processed/sutd_undergrad_faq_all.txt


In [21]:
# Count questions to verify
combined_path = "data/processed/sutd_undergrad_faq_all.txt"

with open(combined_path, "r", encoding="utf-8") as f:
    text = f.read()

num_questions = text.count("--------------")

print("Number of questions:", num_questions)


Number of questions: 83


In [22]:
# Move the old 9 pages (now combined) into archive folder

from pathlib import Path
import shutil

processed_dir = Path("data/processed")
archive_dir = Path("data/archive")
archive_dir.mkdir(parents=True, exist_ok=True)

# move only the individual page files, not the combined one
for file in processed_dir.glob("*_faq.txt"):
    if file.name != "sutd_undergrad_faq_all.txt":
        dest = archive_dir / file.name
        shutil.move(str(file), str(dest))
        print(f"Moved {file.name} → archive/")
        
print("Done.")

Moved admissions_undergraduate_faq_paged_5_faq.txt → archive/
Moved admissions_undergraduate_faq_paged_4_faq.txt → archive/
Moved admissions_undergraduate_faq_paged_6_faq.txt → archive/
Moved admissions_undergraduate_faq_paged_7_faq.txt → archive/
Moved admissions_undergraduate_faq_paged_2_faq.txt → archive/
Moved admissions_undergraduate_faq_paged_3_faq.txt → archive/
Moved admissions_undergraduate_faq_paged_8_faq.txt → archive/
Moved admissions_undergraduate_faq_paged_1_faq.txt → archive/
Moved admissions_undergraduate_faq_paged_9_faq.txt → archive/
Done.


In [23]:
from bs4 import BeautifulSoup
from pathlib import Path
import pandas as pd

RAW_DIR = Path("data/raw")

faq_with_links = []
faq_no_links = []
link_records = []

BASE = "https://www.sutd.edu.sg"

for html_file in RAW_DIR.glob("*.html"):
    soup = BeautifulSoup(html_file.read_text(encoding="utf-8", errors="ignore"), "html.parser")

    accordion = soup.select_one("section#accordion")
    if not accordion:
        continue

    for h6 in accordion.select("h6"):
        question = h6.get_text(" ", strip=True)

        ans_div = h6.find_next("div", class_="richText")
        if not ans_div:
            continue

        # extract answer text
        answer_text = ans_div.get_text("\n", strip=True)

        # extract hyperlinks
        links = []
        for a in ans_div.find_all("a", href=True):
            href = a["href"]
            if href.startswith("/"):
                href = BASE + href
            links.append(href)

        entry = f"--------------\n{question}\n\n{answer_text}\n"

        if links:
            faq_with_links.append(entry)
            for l in links:
                link_records.append({"question": question, "link": l})
        else:
            faq_no_links.append(entry)

# save outputs
Path("data/processed/faq_no_links.txt").write_text("".join(faq_no_links), encoding="utf-8")
Path("data/archive/faq_with_links.txt").write_text("".join(faq_with_links), encoding="utf-8")

pd.DataFrame(link_records).to_csv("data/archive/faq_links_to_visit.csv", index=False)

print("No-link Q&A:", len(faq_no_links))
print("With-link Q&A:", len(faq_with_links))
print("Links extracted:", len(link_records))

No-link Q&A: 44
With-link Q&A: 39
Links extracted: 91


# Q&A Generation

Create question-answer pairs suitable for model training and evaluation. Aim for coverage: admissions eligibility, deadlines, programs, curriculum, scholarships, housing, student life, application process, contact channels.

Options:
- Manual authoring from authoritative sources (preferred for correctness).
- LLM-assisted generation using your curated documents, followed by human validation.

Guidelines:
- Keep answers concise and factual; include references (URL, section) in metadata.
- Avoid speculative or outdated info; include retrieval timestamp.
- Provide diverse phrasings and difficulty levels.


In [ ]:
# Insert code here (if appropriate)

# Dataset Assembly

Format the Q&A into a machine-learning friendly structure (JSONL recommended). Include:
- id, question, answer
- source (URL/file), retrieved_at timestamp
- split (train/test/dev), topic/category

Ensure a clear train/test split with no leakage.


In [ ]:
# Insert code here (if appropriate)

# Prompt-Engineered Prototype

Build a simple chatbot that answers prospective student questions using:
- A concise system prompt (tone: helpful, factual, official).
- Lightweight retrieval from your curated documents for grounding.
- A foundation model API (OpenAI, Anthropic, or AWS Bedrock).

Note: Do not include private keys in the notebook. Use environment variables.


In [ ]:
# Insert code here (if appropriate)

# Evaluation

Evaluate prototype answers against the Q&A dataset. Use a simple metric (e.g., token overlap or string similarity) and manual spot checks.

Suggested metrics:
- Exact match / normalized overlap
- ROUGE-L (optional)
- Human review with rubric (clarity, correctness, completeness, source alignment)


In [ ]:
# Insert code here (if appropriate)

# End

This concludes Individual assignment 1.

Please submit this notebook with your answers and the generated output cells as a **Jupyter notebook file** via github.


Every student should do the following submission steps:
1. Create a private github repository **sutd_5055mlops** under your github user.
2. Add your instructors as collaborator: ddahlmeier, bearwithchris and MarkHershey
3. Save your submission as `individual_assignment_01_StudentID`.ipynb (replace StudentID with your student ID)
4. Push the submission files to your repo 
5. Submit the link to the repo via eDimensions 



**Assignment due 27 Feb (Fri) 23:59**